<a href="https://colab.research.google.com/github/yassir-a-p/blender-on-colab-id/blob/main/blender-on-colab-id.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cek tipe GPU/kartu grafis

In [ ]:
!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv 

#Persiapan
Tentukan versi blender dan *path*/alamat di mana berkas blend yang akan di*render*: upload langsung atau akses file di Google Drive.
Atur dan isi, kemudian jalankan kode.

In [ ]:
#@title Pengaturan Awal
#@markdown Atur versi Blender yang akan dipakai dan pengaturan lainnya

blender_version = 'blender2.92.0' #@param ["blender2.92.0", "blender2.91.2", "blender2.90.1", "blender2.83.13", "blender2.82a", "blender2.81a", "blender2.80", "blender2.79b"] {allow-input: false}
gpu_enabled = True #@param {type:"boolean"}
cpu_enabled = True #@param {type:"boolean"}
#@markdown ---
use_drive = True #@param {type:"boolean"}

## Google Drive
Jalankan kode berikut jika menggunakan Google Drive (mencentang opsi **use_drive** di atas)<br><br>
Klik link yang muncul, lalu akan terbuka tab baru, pilih akun Google Drive yang akan disambungkan, klik **Allow**, *copy* kode verifikasi yang muncul dan *paste* di kotak yang disediakan di bawah ini, lalu tekan **[Enter]**.

In [ ]:
if use_drive:
  from google.colab import drive
  drive.mount('/content/drive')

## Persiapan berkas .blend dan folder hasil render

In [ ]:
#@markdown Klik kanan di file manager (panel samping kiri "Files") lalu copy path dan paste
path_to_blend = '/content/drive/your-folder/your-blend-file.blend'  #@param {type: "string"}
output_path = '/content/drive/your-folder/output/frame_###'  #@param {type: "string"}
#@markdown Tanda pagar ### pada output digunakan untuk secara otomatis menambahkan angka frame pada nama berkas hasil render.

**!!! PERHATIAN !!!**
Jalankan lagi kode di atas setelah mengubah isinya!

# **Proses instalasi Blender pada VM**

**Cukup jalankan kode-kode berikut tanpa perubahan**

Mengarahkan lokasi unduh Blender:

In [ ]:
if blender_version == "blender2.79b":
    download_path="https://ftp.halifax.rwth-aachen.de/blender/release/Blender2.79/blender-2.79b-linux-glibc219-x86_64.tar.bz2"
elif blender_version == "blender2.80":
    download_path="https://ftp.halifax.rwth-aachen.de/blender/release/Blender2.80/blender-2.80-linux-glibc217-x86_64.tar.bz2"
elif blender_version == "blender2.81a":
    download_path="https://ftp.halifax.rwth-aachen.de/blender/release/Blender2.81/blender-2.81a-linux-glibc217-x86_64.tar.bz2"
elif blender_version == "blender2.82":
    download_path="https://ftp.halifax.rwth-aachen.de/blender/release/Blender2.82/blender-2.82a-linux64.tar.xz"
elif blender_version == "blender2.83.13":
    download_path="https://ftp.halifax.rwth-aachen.de/blender/release/Blender2.83/blender-2.83.13-linux64.tar.xz"
elif blender_version == "blender2.90.1":
    download_path="https://ftp.halifax.rwth-aachen.de/blender/release/Blender2.90/blender-2.90.1-linux64.tar.xz"
elif blender_version == "blender2.91.2":
    download_path="https://ftp.halifax.rwth-aachen.de/blender/release/Blender2.91/blender-2.91.2-linux64.tar.xz"
elif blender_version == "blender2.92.0":
    download_path="https://ftp.halifax.rwth-aachen.de/blender/release/Blender2.92/blender-2.92.0-linux64.tar.xz"

Mengunduh, *unpack*, dan memindahkan hasil unpack ke lokasi tertentu:

In [ ]:
!mkdir $blender_version
if blender_version == "blender2.79b" or "blender2.80" or "blender2.81a":
    !wget -O '{blender_version}.tar.bz2' -nc $download_path
    !tar -xf '{blender_version}.tar.bz2' -C ./$blender_version --strip-components=1
else:
    !wget -O '{blender_version}.tar.xz' -nc $download_path
    !tar xf '{blender_version}.tar.xz' -C ./$blender_version --strip-components=1


*Mengatasi weird behaviors yang terjadi dengan libtcmalloc yang muncul pada VM Colab*:

In [ ]:
import os

os.environ["LD_PRELOAD"] = ""

!apt update
!apt remove libtcmalloc-minimal4
!apt install libtcmalloc-minimal4
os.environ["LD_PRELOAD"] = "/usr/lib/x86_64-linux-gnu/libtcmalloc_minimal.so.4.3.0"

!echo $LD_PRELOAD

Dependensi untuk GPU:

In [ ]:
!apt install libboost-all-dev
!apt install libgl1-mesa-dev
!apt install libglu1-mesa libsm-dev

# **Render**

In [ ]:
#@title **Pengaturan Render**

#@markdown Atur **whatToRender** menjadi:<br>
#@markdown + **renderOneFrame** = jika hanya ingin merender 1 frame saja, kemudian isi frame yg diinginkan di bagian _**startFrame**_<br>
#@markdown + **renderAllFrame** = jika ingin render semua frame sesuai yang sudah tersimpan di berkas .blend<br>
#@markdown + **renderWithinRange** = jika ingin render lebih dari 1 frame, dengan rentang frame tertentu. Isi bagian _**startFrame**_ dan _**endFrame**_

whatToRender = 'renderOneFrame' #@param ["renderOneFrame", "renderAllFrame", "renderWithinRange"] {allow-input: false}
#@markdown ---
startFrame =  1#@param {type: "integer"}
endFrame =  250#@param {type: "integer"}
#@markdown ---

#@markdown Centang opsi ini jika ingin menimpa beberapa pengaturan bawaan berkas .blend dengan pengaturan di bawah:
override_certain_settings = False #@param {type:"boolean"}
#@markdown Atur jumlah samples, resolusi, dan compression
samples =  250#@param {type:"integer"}
resolution_x = 1280 #@param {type:"integer"}
resolution_y = 720 #@param {type:"integer"}
compression_level = "100" #@param ["0", "25", "50", "75", "100"] {allow-input: false}
#@markdown **!!! PERHATIAN !!!**<br>
#@markdown Jalankan lagi kode ini setelah mengubah isinya.

# Membuat skrip untuk menimpa beberapa pengaturan dari berkas .blend
override = "# Override Samples, Resolution, dan Compression settings. True samples count, Squared Samples is disabled\n"+\
    "import bpy\n"+\
    "scene = bpy.context.scene\n"+\
    "scene.cycles.use_square_samples = False\n"+\
    "scene.cycles.samples = "+str(samples)+"\n"+\
    "scene.render.resolution_x = "+str(resolution_x)+"\n"+\
    "scene.render.resolution_y = "+str(resolution_y)+"\n"+\
    "scene.render.image_settings.compression = "+str(compression_level)+"\n"
with open('override_others.py', 'w') as f:
    f.write(override)

## Pembuatan skrip

Skrip agar Blender bisa menggunakan GPU untuk CUDA dengan baik

In [ ]:
data = "import re\n"+\
    "import bpy\n"+\
    "scene = bpy.context.scene\n"+\
    "scene.cycles.device = 'GPU'\n"+\
    "prefs = bpy.context.preferences\n"+\
    "prefs.addons['cycles'].preferences.get_devices()\n"+\
    "cprefs = prefs.addons['cycles'].preferences\n"+\
    "print(cprefs)\n"+\
    "# Attempt to set GPU device types if available\n"+\
    "for compute_device_type in ('CUDA', 'OPENCL', 'NONE',):\n"+\
    "    try:\n"+\
    "        cprefs.compute_device_type = compute_device_type\n"+\
    "        print('Device found',compute_device_type)\n"+\
    "        break\n"+\
    "    except TypeError:\n"+\
    "        pass\n"+\
    "#for scene in bpy.data.scenes:\n"+\
    "#    scene.render.tile_x = 64\n"+\
    "#    scene.render.tile_y = 64\n"+\
    "# Enable all CPU and GPU devices\n"+\
    "for device in cprefs.devices:\n"+\
    "    if not re.match('intel', device.name, re.I):\n"+\
    "        print('Activating',device)\n"+\
    "        device.use = "+str(gpu_enabled)+"\n"+\
    "    else:\n"+\
    "        device.use = "+str(cpu_enabled)+"\n"
with open('setgpu.py', 'w') as f:
    f.write(data)

## **Render CYCLES dengan CUDA**

In [ ]:
if override_certain_settings == False:
  if whatToRender == 'renderOneFrame':
    !sudo ./$blender_version/blender -P './setgpu.py' -b '{path_to_blend}' -noaudio -E CYCLES -o '{output_path}' -F 'PNG' -f {startFrame}
  elif whatToRender == 'renderAllFrame':
    !sudo ./$blender_version/blender -P './setgpu.py' -b '{path_to_blend}' -noaudio -E CYCLES -o '{output_path}' -F 'PNG' -a
  elif whatToRender == 'renderWithinRange':
    !sudo ./$blender_version/blender -P './setgpu.py' -b '{path_to_blend}' -noaudio -E CYCLES -o '{output_path}' -F 'PNG' -s {startFrame} -e {endFrame} -a
else:
  if whatToRender == 'renderOneFrame':
    !sudo ./$blender_version/blender -P './setgpu.py' -b '{path_to_blend}' -P './override_others.py' -noaudio -E CYCLES -o '{output_path}' -F 'PNG' -f {startFrame}
  elif whatToRender == 'renderAllFrame':
    !sudo ./$blender_version/blender -P './setgpu.py' -b '{path_to_blend}' -P './override_others.py' -noaudio -E CYCLES -o '{output_path}' -F 'PNG' -a
  elif whatToRender == 'renderWithinRange':
    !sudo ./$blender_version/blender -P './setgpu.py' -b '{path_to_blend}' -P './override_others.py' -noaudio -E CYCLES -o '{output_path}' -F 'PNG' -s {startFrame} -e {endFrame} -a

## **Render dengan OPTIX**

tanpa menjalankan skrip setgpu.py

In [ ]:
if override_certain_settings == False:
  if whatToRender == 'renderOneFrame':
    !sudo ./$blender_version/blender -b '{path_to_blend}' -noaudio -E CYCLES -o '{output_path}' -F 'PNG' -f {startFrame} -- --cycles-device OPTIX 
  elif whatToRender == 'renderAllFrame':
    !sudo ./$blender_version/blender -b '{path_to_blend}' -noaudio -E CYCLES -o '{output_path}' -F 'PNG' -a -- --cycles-device OPTIX 
  elif whatToRender == 'renderWithinRange':
    !sudo ./$blender_version/blender -b '{path_to_blend}' -noaudio -E CYCLES -o '{output_path}' -F 'PNG' -s {startFrame} -e {endFrame} -a -- --cycles-device OPTIX 
else:
  if whatToRender == 'renderOneFrame':
    !sudo ./$blender_version/blender -b '{path_to_blend}' -P './override_others.py' -noaudio -E CYCLES -o '{output_path}' -F 'PNG' -f {startFrame} -- --cycles-device OPTIX 
  elif whatToRender == 'renderAllFrame':
    !sudo ./$blender_version/blender -b '{path_to_blend}' -P './override_others.py' -noaudio -E CYCLES -o '{output_path}' -F 'PNG' -a -- --cycles-device OPTIX 
  elif whatToRender == 'renderWithinRange':
    !sudo ./$blender_version/blender -b '{path_to_blend}' -P './override_others.py' -noaudio -E CYCLES -o '{output_path}' -F 'PNG' -s {startFrame} -e {endFrame} -a -- --cycles-device OPTIX 

---
# **LAMPIRAN**

## Copyright, License and Modifications Notice

    Copyright 2021 Yassir A. P.

    Copyright 2021 github.com/donmahallem

    Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License.

    You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

    This file has been modified to improve workflow, use updated Blender versions, add support for OPTIX rendering, and add options to override settings of Samples, Resolution, and Compression.


## Pertanyaan? Saran? Request?

Jika kalian punya pertanyaan, saran, atau request, silakan menuju [halaman Diskusi repo ini](https://github.com/yassir-a-p/blender-on-colab/discussions). Saya tidak menjanjikan apapun, tapi akan saya coba untuk menjawab pertanyaan yang ada dan mengurus proyek ini. 🙏🏽